In [1]:
from transformers import pipeline,AutoTokenizer, TFCamembertForSequenceClassification
import pandas as pd
from tqdm import tqdm
import numpy as np

df = pd.read_csv('../data/avis/general_df_clean_equil.csv')
df.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-23 18:24:56.869130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,user,etoiles,n_avis,localisation,date_avis,titre_avis,text_avis,date_experience,page
0,27397,64765,8079,19,Sylvain Andujar,3,3.0,FR,2022-07-26 10:10:19,Pour l'instant toujours en attente,"Pour l'instant toujours en attente, tout les j...",2022-07-26,YounitedCredit
1,28396,36538,7640,0,ANNICK JALLET,3,1.0,FR,2022-04-11 14:14:30,intérêts trop élevés,na,2022-04-11,Cofidis
2,26056,18070,1760,0,Client,2,3.0,FR,2023-06-07 16:57:39,"C'était les meilleurs, mais c'était avant",Deux dernières fois où j'ai dû faire appel à F...,2023-06-04,Fortuneo
3,2080,30173,1275,15,Radio Anime,1,1.0,BE,2023-09-08 13:30:04,Vous n'aidez pas les gens,"Vous n'aidez pas les gens, vous faites l'inver...",2023-09-08,Cofidis
4,154,54340,781,1,Serge Péon,1,2.0,FR,2023-02-10 20:32:37,De voyage à l'étranger mon conseillé de…,De voyage à l'étranger mon conseillé de la Ban...,2023-02-09,MaFrenchBank


In [2]:
# chargement du modèle
tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine")
model = TFCamembertForSequenceClassification.from_pretrained("tblard/tf-allocine")

classifier=  pipeline("text-classification", model = model, tokenizer= tokenizer)
#tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512,'return_tensors':'pt'}

2023-10-23 18:25:23.745512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.


In [30]:
outputs = classifier(df['text_avis'][0])
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.95979


In [3]:
MAX_LENGTH = 512  # nombre de mots max pour Camembert

sentiment = pd.DataFrame()
df['label'] =np.nan
df['score'] = np.nan
for i, text in tqdm(enumerate(df.text_avis), total=df.shape[0]):
    
    # Tronquer le texte s'il est trop long
    text = text[:MAX_LENGTH]

    try:
        temp = classifier(text)
        temp = pd.DataFrame(temp)
        df['label'][i] = temp['label'][0]
        df['score'][i] = temp['score'][0]
    except Exception as e:
        print(f"Erreur lors du traitement du texte à l'indice {i}. Erreur: {e}")

  0%|          | 0/15050 [00:00<?, ?it/s]/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_34413/3276902870.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = temp['label'][0]
/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_34413/3276902870.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'][i] = temp['score'][0]
  0%|          | 1/15050 [00:00<1:20:27,  3.12it/s]/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_34413/3276902870.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

KeyboardInterrupt: 

In [27]:
# transformation des scores pour intégrer la polarité et la confiance du modèle
# vers -1 est très probablement négativ, vers +1 est très probablement positif
df['label'] = df.label.replace("NEGATIVE", -1)
df['label'] = df.label.replace("POSITIVE", 1)
df['sentiment_norm'] = df["label"] * df["score"]
df.head()

/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_10307/304819803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['label'] = df_small.label.replace("NEGATIVE", -1)
/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_10307/304819803.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['label'] = df_small.label.replace("POSITIVE", 1)
/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_10307/304819803.py:5: SettingWithCopyWarning: 
A value is trying to be set on a co

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,user,etoiles,n_avis,localisation,date_avis,titre_avis,text_avis,date_experience,page,label,score,sentiment_norm
0,0,0,0,Mme Rakotondrasamy Lalanirina,1,3.0,FR,2023-10-20 03:16:24,Orgueil gestionnaire,"Ça reste une mauvaise expérience, quand ton ge...",2023-10-19,MaBanqueBNPParibas,-1.0,0.959790,-0.959790
1,155493,756,16,Mathieu,1,5.0,FR,2023-05-09 15:16:41,Délais ridicules,Ça fait 2 jours que je souhaite modifier mon p...,2023-05-08,OrangeBank,-1.0,0.786694,-0.786694
2,26631,4636,16,Hassan,1,5.0,FR,2018-04-26 19:44:47,Je suis très déçu du service fourni par…,Je suis très déçu du service fourni par Anytim...,2018-04-26,Anytime,-1.0,0.876774,-0.876774
3,54607,1048,8,Oriol Serrando,1,4.0,ES,2022-09-14 11:11:11,Rien ne va,Rien ne va: plus cher et beaucoup moins bien q...,2022-09-14,MaFrenchBank,-1.0,0.998726,-0.998726
4,54609,1050,10,Sasa Sasa,1,2.0,FR,2022-09-06 17:45:27,"Très déçue, préférez plutôt Hello Bank",Très déçue. Partie aux USA il y a eu plusieurs...,2022-07-01,MaFrenchBank,-1.0,0.998746,-0.998746


In [ ]:
df['text_length'] = len

In [ ]:
df.to_csv("../data/avis/general_df_clean_sent.csv")

In [12]:
#df['longueur_text'] = [lambda x: len(df.text_avis[x])]

for i, text in enumerate(df.text_avis):
    df['longueur_text'][i] = len(df.text_avis[i])

/var/folders/7m/gxswx1js22x0hszgmzpd29qcg1657r/T/ipykernel_34413/1730476520.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['longueur_text'][i] = len(df.text_avis[i])


0        138
1          2
2        289
3        189
4        478
        ... 
15045    458
15046     54
15047    337
15048      2
15049    394
Name: longueur_text, Length: 15050, dtype: object